############################
##- Installing libraries -##
#--------- Python 3 -------#
############################
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain

def install(package):
    pipmain(['install', "--upgrade", "pip"])
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        try:
            pip._internal.main(['install', package])
        except:
            print(pip._internal.main(['install', package]))

###########################
##- Installing Biopython-##
###########################
if _name_ == '_main_':
    install('biopython')

In [2]:
############################
##- Installing libraries -##
##------- Python 2 -------##
############################
import pip

def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])

###########################
##- Installing Biopython-##
###########################
if __name__ == '__main__':
    install('biopython')

In [38]:
##- Importing linbraries -##
############################
import os
import glob
import subprocess

#- Setting the working directory -##
####################################
path = '/home/anajulia/Be_project/fastasequences'
os.chdir(path)

In [12]:
#- Spliting sequences into separates files -##
##############################################
files_list = glob.glob('*.txt')
for i in files_list:
    subprocess.call(['split','-l2','-d',i,i.split('_')[0]+'_sequence.'])

In [45]:
##- Importing libraries -##
############################
from Bio import pairwise2
from Bio import SeqIO
from Bio.Phylo.TreeConstruction import DistanceCalculator
try:
    from StringIO import StringIO # Python 2
except ImportError:
    from io import StringIO # Python 3
import itertools

#- reading files -##
####################
all_files = os.listdir(path)
files_list = glob.glob('*.txt')

#- Pairwise aligments all to all -##
####################################
for i in files_list:
    ped_code = i.split('_')[0]
    seqs_list = []
    for j in all_files:
        if j.startswith(ped_code):
            if j.endswith('.txt') == False:
                seqs_list.append(j)
                
    combinations = list(itertools.combinations(seqs_list, 2))
    if len(combinations) > 1:
        for x in combinations:
            ##-- we take only the combinations including the uniprot seq ref
            if '00' in str(x):
                #######-- Aligning sequences --########
                data1 = SeqIO.read(x[0], "fasta")
                data2 = SeqIO.read(x[1], "fasta")
                ##-- Align --##
                alignments = pairwise2.align.globalxx(data1.seq,data2.seq)
                ##-- Ploting aligments --##
                aln = pairwise2.format_alignment(*alignments[0])
                #-- Building an output --#
                path_out = path +'/'+ x[0].split('_')[0]+x[0].split('_')[-1][-2:] +'-'+ x[1].split('_')[0] + x[1].split('_')[-1][-2:] +'_aln_pair.plot'
                path_out_fasta = path +'/'+ x[0].split('_')[0]+x[0].split('_')[-1][-2:] +'-'+ x[1].split('_')[0] + x[1].split('_')[-1][-2:] +'_pair.fasta'
                print(path_out_fasta)
                ##-- Writing outputs --##
                #with open(path_out,'a') as miarch:
                    #miarch.write(aln)

/home/anajulia/Be_project/fastasequences/PED3AAB02-PED3AAB00_pair.fasta
/home/anajulia/Be_project/fastasequences/PED3AAB01-PED3AAB00_pair.fasta
/home/anajulia/Be_project/fastasequences/PED3AAB03-PED3AAB00_pair.fasta
/home/anajulia/Be_project/fastasequences/PED1AAA00-PED1AAA03_pair.fasta
/home/anajulia/Be_project/fastasequences/PED1AAA00-PED1AAA02_pair.fasta
/home/anajulia/Be_project/fastasequences/PED1AAA00-PED1AAA01_pair.fasta
/home/anajulia/Be_project/fastasequences/PED3AAA02-PED3AAA00_pair.fasta
/home/anajulia/Be_project/fastasequences/PED3AAA01-PED3AAA00_pair.fasta
/home/anajulia/Be_project/fastasequences/PED3AAA00-PED3AAA03_pair.fasta
/home/anajulia/Be_project/fastasequences/PED4AAA00-PED4AAA01_pair.fasta
/home/anajulia/Be_project/fastasequences/PED4AAA00-PED4AAA03_pair.fasta
/home/anajulia/Be_project/fastasequences/PED4AAA00-PED4AAA02_pair.fasta


In [ ]:
#########################################
##-- Moving files to a new directory --##
#########################################
dirName = 'pairwise_alig_plot'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " + dirName + " Created ") 
except FileExistsError:
    print("Directory " + dirName +  " already exists")

###########################
##-- Moving plot files --##    
source = path +'/'
dest1 = path + '/' + dirName

##-- Listing files --##
files = glob.glob('*.plot')
##-- Moving files --##
import shutil
for f in files:
        shutil.move(source + f, dest1)
print('All the plot files were moved correctly')

In [21]:
##- Importing libraries -##
############################
import itertools
import os
import subprocess

In [30]:
###################################################
##-- checking if clustalw is already installed --##
#  Source: https://askubuntu.com/questions/35744  #
###################################################

##-- def query_package --##
import os.path
path = '/usr/bin/clustalw'
def query_package(path):
    if os.path.exists(path):
        return 'clustalw is ready to use'
    else:
        return 'please install clustalw'

##-- def _install_clustalw --##
def _install_clustalw():
    subprocess.call("sudo get install clustalw", shell=True)

##-- check point --##
if query_package(path) == "clustalw is ready to use":
    print('clustalw is ready to use')
    pass
else:
    print('Installing clustalw')
    _intall_clustal()
    print('Done')

clustalw is ready to use


In [36]:
##-- Listing fasta files --##
list_fasta_seq = glob.glob('*.txt')

##-- Runing clustal --##
for i in list_fasta_seq:

<open file 'PED2AAB_all_sequence.txt', mode 'rU' at 0x7f1ab7888db0>
<open file 'PED3AAB_all_sequence.txt', mode 'rU' at 0x7f1ab78230c0>
<open file 'PED1AAA_all_sequence.txt', mode 'rU' at 0x7f1ab7888db0>
<open file 'PED2AAA_all_sequence.txt', mode 'rU' at 0x7f1ab78230c0>
<open file 'PED2AAD_all_sequence.txt', mode 'rU' at 0x7f1ab7888db0>
<open file 'PED1AAD_all_sequence.txt', mode 'rU' at 0x7f1ab78230c0>
<open file 'PED1AAB_all_sequence.txt', mode 'rU' at 0x7f1ab7888db0>
<open file 'PED3AAA_all_sequence.txt', mode 'rU' at 0x7f1ab78230c0>
<open file 'PED3AAD_all_sequence.txt', mode 'rU' at 0x7f1ab7888db0>
<open file 'PED4AAA_all_sequence.txt', mode 'rU' at 0x7f1ab78230c0>
